In [1]:
import tensorflow as tf

c:\users\subhash nerella\appdata\local\conda\conda\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./cs231n/datasets/MNIST_data', one_hot=True)

Extracting ./cs231n/datasets/MNIST_data\train-images-idx3-ubyte.gz
Extracting ./cs231n/datasets/MNIST_data\train-labels-idx1-ubyte.gz
Extracting ./cs231n/datasets/MNIST_data\t10k-images-idx3-ubyte.gz
Extracting ./cs231n/datasets/MNIST_data\t10k-labels-idx1-ubyte.gz


In [3]:
def conv_layer(input, size_in, size_out, name="conv"):
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([5, 5, size_in, size_out], stddev=0.1), name="W")
        b = tf.Variable(tf.constant(0.1, shape=[size_out]), name="B")
        conv = tf.nn.conv2d(input, w, strides=[1, 1, 1, 1], padding="SAME")
        act = tf.nn.relu(conv + b)
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", act)
        return tf.nn.max_pool(act, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

In [4]:
def fc_layer(input, size_in, size_out, name="fc"):
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([size_in, size_out], stddev=0.1), name="W")
        b = tf.Variable(tf.constant(0.1, shape=[size_out]), name="B")
        act = tf.matmul(input, w) + b
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", act)
        return act

In [5]:
tf.reset_default_graph()
sess = tf.Session()

In [6]:
x = tf.placeholder(tf.float32,shape=[None,784],name="X")
x_image = tf.reshape(x,[-1,28,28,1])
tf.summary.image('input', x_image, 3)
y = tf.placeholder(tf.float32, shape=[None, 10], name="labels")

In [7]:
conv1 = conv_layer(x_image,1,32,"conv1")
conv2 = conv_layer(conv1,32,64,"conv2")
flattened = tf.reshape(conv2,[-1,7*7*64])
fc1 = fc_layer(flattened,7*7*64,1024,"fc1")
relu = tf.nn.relu(fc1)
tf.summary.histogram("fc1/relu", relu)
with tf.name_scope("dropout"):
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(relu, keep_prob)

logits = fc_layer(h_fc1_drop, 1024, 10, "fc2")

In [8]:
with tf.name_scope("xent"):
    xent = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(
            logits=logits, labels=y), name="xent")
    tf.summary.scalar("xent", xent)

In [9]:
with tf.name_scope("train"):
    train_step = tf.train.AdamOptimizer(1e-4).minimize(xent)

with tf.name_scope("accuracy"):
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar("accuracy", accuracy)

In [10]:
summ = tf.summary.merge_all()
sess.run(tf.global_variables_initializer())
writer = tf.summary.FileWriter("/assignment2/4")
writer.add_graph(sess.graph)

In [11]:
for i in range(8001):
    batch = mnist.train.next_batch(100)
    if i % 100 == 0:
        [train_accuracy, s] = sess.run([accuracy, summ], feed_dict={x: batch[0], y: batch[1],keep_prob: 1.0})
        writer.add_summary(s, i)
        print('step %d, training accuracy %g' % (i, train_accuracy))
    sess.run(train_step, feed_dict={x: batch[0], y: batch[1],keep_prob: 0.5})



step 0, training accuracy 0.07
step 100, training accuracy 0.86
step 200, training accuracy 0.92
step 300, training accuracy 0.95
step 400, training accuracy 0.87
step 500, training accuracy 0.97
step 600, training accuracy 0.95
step 700, training accuracy 0.96
step 800, training accuracy 0.99
step 900, training accuracy 0.97
step 1000, training accuracy 0.97
step 1100, training accuracy 0.97
step 1200, training accuracy 0.97
step 1300, training accuracy 0.99
step 1400, training accuracy 0.94
step 1500, training accuracy 0.98
step 1600, training accuracy 0.98
step 1700, training accuracy 0.98
step 1800, training accuracy 0.98
step 1900, training accuracy 0.96
step 2000, training accuracy 0.97
step 2100, training accuracy 0.97
step 2200, training accuracy 0.97
step 2300, training accuracy 0.98
step 2400, training accuracy 1
step 2500, training accuracy 1
step 2600, training accuracy 0.98
step 2700, training accuracy 0.99
step 2800, training accuracy 0.97
step 2900, training accuracy 0.9

In [12]:
batch = mnist.test.next_batch(1000)
print('test accuracy %g' % sess.run(accuracy,feed_dict={
      x: batch[0], y: batch[1], keep_prob: 0.5}))

test accuracy 0.984
